##### 2021/07
###### author : 손해원

- 인스타그램에 키워드를 검색하여 게시글의 정보를 크롤링하는 코드입니다. 

In [2]:
from selenium import webdriver
import time
import re
import json
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import os
import multiprocessing

### Load Chromedriver & Log In

In [3]:
my_id = 'yeonhhhhh5'   ### ID 
my_pw = '@aa1149118'   ### PASSWORD
driver_path = 'C:\dev_python\Webdriver\chromedriver'
instagram ='https://www.instagram.com'
# id/pw 입력 -> 로그인 정보 저장 여부 -> 피드

In [4]:
driver = webdriver.Chrome(driver_path)
driver.get(instagram)

<ipython-input-4-2328cdddebf4>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [5]:
time.sleep(2)
id_ = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input') # id
id_.send_keys(my_id)
pw_ = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input') # pw
pw_.send_keys(my_pw)
driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3)').click() # 로그인


<ipython-input-5-26b57dbbb05f>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id_ = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input') # id
<ipython-input-5-26b57dbbb05f>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pw_ = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input') # pw
<ipython-input-5-26b57dbbb05f>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3)').click() # 로그인


In [6]:
time.sleep(3)
driver.find_element_by_class_name('sqdOP.yWX7d.y3zKF').click() # 정보 저장 안함
    
driver.wait = driver.implicitly_wait(10)

# 알림 설정 나중에 
driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()
    

<ipython-input-6-31d05c215dbb>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('sqdOP.yWX7d.y3zKF').click() # 정보 저장 안함
<ipython-input-6-31d05c215dbb>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()


In [7]:
query = '김천호텔'   ######################## 검색할 내용 입력

try:
    url = 'https://www.instagram.com/explore/tags/{}/'.format(query)
    driver.get(url)
except :
    time.sleep(3)
    driver.get(url) 
    
driver.maximize_window()

In [8]:
# 첫번째 게시물 클릭
first_img = '_9AhH0'
driver.find_element_by_class_name(first_img).click()

<ipython-input-8-0c229527d784>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name(first_img).click()


### 게시물 스크랩

In [9]:
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import requests
import shutil

In [10]:
# data lists
cnt = []               #Content_Counter 
content_links = []     #Content_Link 
user_ids = []          
location_infos = []
location_hrefs = []
main_texts = []
tags= []
likes = []
who_likes = []
date_texts = []
date_times = []
date_titles = []

tag_lists = []
img_list = []


# path 설정
user_id_path = 'e1e1d'
location_class = 'O4GlU'
main_text_path = 'C4VMK'
tags_class_name = ' xil3i'
likes_path = 'zV_Nj'
date_path = '_1o9PC.Nzb55'
next_path = '_65Bje.coreSpriteRightPaginationArrow'


wish_num = 700# 스크랩할 게시물 개수 설정 
num = 0


for num in tqdm(range(wish_num)):

    # sequence
    cnt.append(num)

    # 글 링크
    try:
        time.sleep(1)
        content_link = driver.find_element_by_class_name('zV_Nj').get_attribute('href')
        content_links.append(content_link)
    except:
        content_links.append("NULL")

    # user ID
    try :
        user_id_obj = driver.find_element_by_class_name(user_id_path)
        user_id = user_id_obj.text

        user_ids.append(user_id)

    except :
        user_ids.append("NULL")


    # 위치정보
    try :
        location_obj = driver.find_element_by_class_name(location_class)
        location_text = location_obj.text
        location_href = location_obj.get_attribute("href")

        location_infos.append(location_text)
        location_hrefs.append(location_href)
    except :
        location_infos.append("NULL")
        location_hrefs.append("NULL")


    # 이미지 링크 
    try:
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        instagram = soup.select('.v1Nh3.kIKUG._bz0w')
        img_url = instagram[num].select_one('.KL4Bh').img['src']
        img_list.append(img_url)
        

    except:
        img_list.append("NULL")
        

    # 본문
    try :
        main_text_obj = driver.find_element_by_class_name(main_text_path)
        main_text = main_text_obj.text
        clean_text = main_text.replace("\n"," ")
        #comments = driver.find_element_by_class_name('Mr508 ').text.replace('\n',' ')

        main_texts.append(clean_text)

    except :
        main_texts.append("NULL")

    # 해쉬태그
    try:
        tag_list = []
        tags = driver.find_elements_by_class_name('xil3i')
        for i in range(len(tags)):
            tag = tags[i].text.replace('#', '')
            tag_list.append(tag)
        tag_lists.append(tag_list)
    except:
        tag_list.append("NULL")

    # 좋아요 개수
    try :
        time.sleep(2)
        like = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/section[2]/div/div/a/span')
    
        likee = like.text
        likes.append(likee)
    except :
        likes.append(None)


    # 날짜
    #try :
    date_obj = driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time')
    
    date_text = date_obj.get_attribute("title")
        
    date_texts.append(date_text)
        #date_time = date_obj.get_attribute("datetime")
        #date_title = date_obj.get_attribute("title") # 2021년 7월 14일
        
        #date_times.append(date_time)
        #date_titles.append(date_title)
   # except :
        #date_texts.append("NULL")
        #date_times.append("NULL")
        #date_titles.append("NULL")


    print(" ")
    print(f"-----{num}-----")
    print("ID: ", user_ids[num])
    print("위치정보: ", location_infos[num])
    print("내용: ", main_texts[num])
    print("날짜: ", date_texts[num])
    print(" 좋아요개수 :",likes[num])
    print("   ")


    # 다음 페이지 여부 확인
    try:
        next_level = driver.find_element_by_class_name('l8mY4 ')
        next_level.click()
        print('clicked')
        num += 1
    except:
        break
        


  0%|                                                                                          | 0/700 [00:00<?, ?it/s]<ipython-input-10-7349819d2929>:41: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content_link = driver.find_element_by_class_name('zV_Nj').get_attribute('href')
<ipython-input-10-7349819d2929>:48: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  user_id_obj = driver.find_element_by_class_name(user_id_path)
<ipython-input-10-7349819d2929>:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  location_obj = driver.find_element_by_class_name(location_class)
<ipython-input-10-7349819d2929>:86: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  main_text_obj = driver.find_element_by_class_name(main_text_path)
<ipython-input-10-7349819d2929>:99: DeprecationWarning: find_element

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div/article/div/div[2]/div/div/div[2]/div[2]/a/time"}
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x006106F3+2492147]
	Ordinal0 [0x005A9BD1+2071505]
	Ordinal0 [0x004B2478+1057912]
	Ordinal0 [0x004DC964+1231204]
	Ordinal0 [0x00506B62+1403746]
	Ordinal0 [0x004F57FA+1333242]
	Ordinal0 [0x00504F38+1396536]
	Ordinal0 [0x004F568B+1332875]
	Ordinal0 [0x004D21D4+1188308]
	Ordinal0 [0x004D302F+1191983]
	GetHandleVerifier [0x007967A6+1545030]
	GetHandleVerifier [0x0084105C+2243580]
	GetHandleVerifier [0x0069BC97+518199]
	GetHandleVerifier [0x0069AD80+514336]
	Ordinal0 [0x005AED2D+2092333]
	Ordinal0 [0x005B2EE8+2109160]
	Ordinal0 [0x005B3022+2109474]
	Ordinal0 [0x005BCB71+2149233]
	BaseThreadInitThunk [0x75E26359+25]
	RtlGetAppContainerNamedObjectPath [0x77E187A4+228]
	RtlGetAppContainerNamedObjectPath [0x77E18774+180]


In [ ]:
len(date_titles)

In [ ]:
instagram_df=pd.DataFrame(
    {
        "cnt": cnt,
        "link":content_links,
        "id": user_ids
        , "location_info": location_infos
        , "location_href": location_hrefs
        , "main_text": main_texts
        , "hastag" : tag_lists
        , "like": likes
        #, "date_time": date_times
        , "date_title": date_texts
    }
)

In [ ]:
# dfdf = pd.concat([instagram_df_1, instagram_df_2] , axis=0)

# instagram_df_ok = pd.concat([dfdf, instagram_df_3], axis=0)

In [ ]:
instagram_df.all()

In [ ]:
#instagram_df.to_csv('./instagram_seoul_1102.csv', index=False)

#### 좋아요 목록의 유저 링크 가져오기

In [ ]:
###### 0831 완셩

def get_likers():
    real_list = []

    for num in tqdm(range(len(content_links))): # 컨텐츠 링크를 읽어들여 그 수만큼 반복

        url = content_links[num]
        driver.get(url)

        like_obj = driver.find_element_by_class_name('zV_Nj').text[4:-1].replace(',','') # 좋아요 수 구하기 
        print('좋아요: ',like_obj)
    #     like_float = float(like_obj)
    #     like = int(like_obj)

        time.sleep(1)
        driver.find_element_by_class_name('zV_Nj').click() # 좋아요 클릭 
        time.sleep(3)

        likers_list = []
        likers_dic = {}

        likers = driver.find_elements_by_class_name('FPmhX.notranslate.MBL3Z') 
        for j in tqdm(range(len(likers))):
            likerss = driver.find_elements_by_class_name('FPmhX.notranslate.MBL3Z')
            time.sleep(3)

            print(likerss[j].text)
            link_info = likerss[j].get_attribute('href') # 링크

            likers_list.append(link_info)
            likers_dic = likers_list



            j += 1

            if j % 6 == 0 :

                 # 페이지 다운
                endOfPage = driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div').find_elements_by_tag_name('a')[-1]
                endOfPage.send_keys(Keys.PAGE_DOWN)
                time.sleep(3)
                continue

        real_list.append(likers_dic)


        num += 1
        
    like_df = pd.DataFrame(
        {
        "content_link":content_links,
        "likers":real_list
        }
    )

    like_df.to_csv('./liker_vanlialatte')

In [ ]:
# 각 유저의 프로필 정보 -> id가 있는 경우 

followers_count = []
gesimul_count =[]
sogaes = []

    
def get_profile(user_ids):

    
    i = 0
    for i in range(len(user_ids)):
        user_profile = 'https://www.instagram.com/{}/'.format(user_ids[i])
        driver.get(user_profile)

        try: 

            gesimul = int(driver.find_element_by_class_name('g47SY ').text.replace(',',''))

            followers = \
            int(driver.find_element_by_css_selector('#react-root > section > main > div > header > section > ul > li:nth-child(2) > a > span').get_attribute("title").replace(',',''))

            sogae = driver.find_element_by_class_name('-vDIg').text.replace('\n',' ')
        #     profiles.append(user_ids[i])
            gesimul_count.append(gesimul)
            followers_count.append(followers)
            sogaes.append(sogae)
            
            print("ID: ", user_ids[i])
            print("게시물: ",gesimul)
            print("팔로워: ",followers)
            print("소개: ",sogae)
            print(" ")

            i += 1
        except :
            followers_count.append("NaN")
            gesimul_count.append("NaN")
            sogaes.append("NaN")
            
        


In [ ]:
get_profile(user_ids)

In [ ]:
print(len(user_ids))
print(len(location_infos))
print(len(location_hrefs))
print(len(main_texts))
print(len(tags))
print(len(likes))
print(len(date_texts))
print(len(date_times))
print(len(date_titles))

### DataFrame

In [ ]:
instagram_df = pd.DataFrame(
    {
         "id" : user_ids
        ,"location_info" : location_infos
        ,"location_href" : location_hrefs
        ,"main_text" : main_texts
        ,"hastag" : tag_lists
        ,"like" : likes
       ,"date_text" : date_texts
        #,"date_time" : date_times
       #,"date_title" : date_titles
    }
)

In [ ]:
user_df = pd.DataFrame(
    {
          "user_id": user_ids
        , "Posts": gesimul_count
        , "Followers": followers_count
        , "user_profile" : sogaes
    }
)

In [ ]:
user_df.drop_duplicates(inplace=True)

In [ ]:
user_df.head()

In [ ]:
instagram_df.head(10)

### Concat

In [ ]:
result = pd.concat([instagram_df,user_df], axis=1)
result

## To_csv

In [ ]:
file_name = 'instagrame_1110_김천호텔(700)' ##### 파일명
file_path = 'C:/Users/seoyeonho/Desktop/ERC_LAB/김천호텔_/' ### 저장할 경로 

In [ ]:
result.to_csv('{}{}.csv'.format(file_path,file_name), index=False, encoding='utf-8')

### Read

In [ ]:
pd.read_csv('{}{}.csv'.format(file_path,file_name))

In [ ]:
instagram_df.to_csv('{}'.format("0908.csv"), index=False, encoding='utf-8')

In [ ]:
instagram_df